# Imports

In [58]:
#Standard Packages
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import pickle
import warnings
warnings.filterwarnings(action='ignore') 

# Packages used for API calls and data processing
import requests
import json
def get_keys(path):
    with open(path) as f:
        return json.load(f)
import ast
import time
import http.client, urllib.request, urllib.parse, urllib.error, base64
api_key = 'ceeaacb7cf024c7485e00ef8457e42dc'
gamertag = 'Drymander'
from tqdm import tqdm
# !pip install isodate
import isodate
import streamlit as st

In [59]:
# Preprocessing tools
from sklearn.model_selection import train_test_split,cross_val_predict,cross_validate
from sklearn.preprocessing import MinMaxScaler,StandardScaler,OneHotEncoder
scaler = StandardScaler()
from sklearn import metrics

# Models & Utilities
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder


# Streamlit Function Save

In [60]:
streamlit_function_save = True

# Functions

## Gamertag for API

In [83]:
# %%writefile 'gamertag_for_api.py'

# Prepare gamertag for API
def gamertag_for_api(gamertag):
    
    # Replace spaces with '+'
    gamertag = gamertag.replace(' ','+')
    return gamertag

# Testing the function
# gamertag_for_api('this is a test')

Overwriting gamertag_for_api.py


## Pull Recent Match

In [168]:
# %%writefile 'pull_recent_match.py'
api_key = 'ceeaacb7cf024c7485e00ef8457e42dc'
# Function to pull most recent match stats into JSON format
# Uses two separate API calls, one from player history and another from match details
def pull_recent_match(gamertag, how_recent, explore=False):
    
    # Use gamertag_for_api function to remove any spaces
    gamertag = gamertag_for_api(gamertag)
    headers = {
        # Request headers
        'Ocp-Apim-Subscription-Key': 'ceeaacb7cf024c7485e00ef8457e42dc',
    }
    # Pulls from arena mode, how_recent is how far to go back in the match history
    # 'count' refers to the number of matches to pull
    params = urllib.parse.urlencode({
        # Request parameters
        'modes': 'arena',
        'start': how_recent,
        'count': 1,
        'include-times': True,
    })
    
    # Try this, otherwise return error message
    try:
        
        # Connect to API and pull most recent match for specified gamer
        conn = http.client.HTTPSConnection('www.haloapi.com')
        conn.request("GET", f"/stats/h5/players/{gamertag}/matches?%s" % params, "{body}", headers)
        response = conn.getresponse()
        latest_match = json.loads(response.read())
        
        # Identify match ID and match date
        match_id = latest_match['Results'][0]['Id']['MatchId']
        match_date = latest_match['Results'][0]['MatchCompletedDate']['ISO8601Date']
        
        # Rest for 1.01 seconds to not get blocked by API
        time.sleep(1.01)
        
        # Using match_id, pull details from match
        conn.request("GET", f"/stats/h5/arena/matches/{match_id}?%s" % params, "{body}", headers)
        response = conn.getresponse()
        data = response.read()
        
        # Option to return as byte string for alternative viewing
        if explore == True:
            print(data)
        else:
            # Append match ID and date from player history API call
            match_results = json.loads(data)
            match_results['MatchId'] = match_id
            match_results['Date'] = match_date
        conn.close()
    
    # Print error if issue with calling API
    except Exception as e:
        print(f"[Errno {0}] {1}".format(e.errno, e.strerror))
    
    # Return match results as JSON
    return match_results

# Show result
match_results = pull_recent_match('Drymander', 0, explore=False)
# match_results

Overwriting pull_recent_match.py


## Build Base Dataframe

In [159]:
# %%writefile 'build_base_dataframe.py'

# Function to build the base dataframe for a single match
# Designed to take in the JSON provided by the pull_recent_match function
def build_base_dataframe(match_results, gamertag):
    
    # Build empty base match dataframe
    df = pd.DataFrame()
    columns = [
        'Finished'
        'TeamId',
        'Gamertag',
        'SpartanRank',
        'PrevTotalXP',
    ]
    df = pd.DataFrame(columns = columns)
    
    # Populate base match dataframe with player stats for each player
    i = 0
    for player in match_results['PlayerStats']:

        player_dic = {}
        # Team ID
        player_dic['DNF'] = match_results['PlayerStats'][i]['DNF']
        player_dic['TeamId'] = match_results['PlayerStats'][i]['TeamId']
        # Team Color
        player_dic['TeamColor'] = match_results['PlayerStats'][i]['TeamId']
        # Gamer Tag
        player_dic['Gamertag'] = match_results['PlayerStats'][i]['Player']['Gamertag']
        # Spartan Rank
        player_dic['SpartanRank'] = match_results['PlayerStats'][i]['XpInfo']['SpartanRank']
        # Previous Total XP
        player_dic['PrevTotalXP'] = match_results['PlayerStats'][i]['XpInfo']['PrevTotalXP']
        df = df.append(player_dic, ignore_index=True)
        i += 1
    
    ########## DATE, GAME VARIANT, MAP ID, MATCH ID, PLAYLIST ID ##########
    df['Date'] = match_results['Date']
    df['Date'] = pd.to_datetime(df['Date']).dt.tz_convert(None)
#     df['Date'] = df['Date'].floor('T')
    df['MatchId'] = match_results['MatchId']
    df['GameBaseVariantId'] = match_results['GameBaseVariantId']
    df['MapVariantId'] = match_results['MapVariantId']
    df['PlaylistId'] = match_results['PlaylistId']
    
    ########## DEFINE PLAYER TEAM ##########
    playerteam = df.loc[df['Gamertag'] == gamertag, 'TeamId'].values[0]
    if playerteam == 0:
        enemyteam = 1   
    else:
        enemyteam = 0
        
    df['PlayerTeam'] = df['TeamId'].map({playerteam:'Player', enemyteam:'Enemy'})
    
    if match_results['TeamStats'][0]['TeamId'] == playerteam:
        playerteam_stats = match_results['TeamStats'][0]
        enemyteam_stats = match_results['TeamStats'][1]
    else: 
        playerteam_stats = match_results['TeamStats'][1]
        enemyteam_stats = match_results['TeamStats'][0]
    
    ########## DETERMINE WINNER ##########
    # Tie
    if playerteam_stats['Rank'] == 1 and enemyteam_stats['Rank'] == 1:
        df['Winner'] = 'Tie'
    # Player wins
    elif playerteam_stats['Rank'] == 1 and enemyteam_stats['Rank'] == 2:
        df['Winner'] = df['TeamId'].map({playerteam:'Victory', enemyteam:'Defeat'})
    # Enemy wins
    elif playerteam_stats['Rank'] == 2 and enemyteam_stats['Rank'] == 1:
        df['Winner'] = df['TeamId'].map({enemyteam:'Victory', playerteam:'Defeat'})
    # Error handling
    else:
        winner = 'Error determining winner'
    
    ########## TEAM COLOR ##########
    df['TeamColor'] = df['TeamId'].map({0:'Red', 1:'Blue'})
    
    # Set columns
    df = df[['Date', 'MatchId', 'GameBaseVariantId', 'PlaylistId', 'MapVariantId', 'DNF',
             'TeamId', 'PlayerTeam', 'Winner', 'TeamColor', 
             'Gamertag', 'SpartanRank', 'PrevTotalXP',
            ]]
    # Sort match by winning team
    df = df.sort_values(by=['Winner'], ascending=False)
    
    return df

# df = build_base_dataframe(pull_recent_match(8), 'Drymander')

# df

Overwriting build_base_dataframe.py


## Get Player List

In [162]:
%%writefile 'get_player_list.py'

# Function to combine all gamertags from the match and prepare them in string
# format for the next API call
def get_player_list(df):
    
    # Create list from our df['Gamertag'] column and remove the brackets
    player_list = str(list(df['Gamertag']))[1:-1]
    
    # Format string for API
    player_list = player_list.replace(', ',',')
    player_list = player_list.replace("'",'')
    player_list = player_list.replace(' ','+')
    
    # Return in one full string
    return player_list

# get_player_list(df)

Overwriting get_player_list.py


## Get Player History

In [165]:
# %%writefile 'get_player_history.py'

# Function to pull more informative information about each player in the match
# This information is not available in the two previous API calls
def get_player_history(df, readable=False):
    headers = {
        # Request headers
        'Ocp-Apim-Subscription-Key': 'ceeaacb7cf024c7485e00ef8457e42dc',
    }
    params = urllib.parse.urlencode({
    })
    # Use our function in the block above the prepare the gamertags for the API
    player_list_api = get_player_list(df)
    
    # Try calling service records API using our player list
    try:
        conn = http.client.HTTPSConnection('www.haloapi.com')
        conn.request("GET", f"/stats/h5/servicerecords/arena?players={player_list_api}&%s" % params, "{body}", headers)
        response = conn.getresponse()
        data = response.read()
        player_history = json.loads(data)
        conn.close()
    
    # Return error if issue with API
    except Exception as e:
        print(f"[Errno {0}] {1}".format(e.errno, e.strerror))
    
    # Option to view in byte string readable format
    if readable == False:
        return player_history
    else:
        return data

# Show result
# player_history = get_player_history(df)
# player_history

Overwriting get_player_history.py


## Build History Dataframe

In [164]:
# %%writefile 'build_history_dataframe.py'

# Function to build secondary dataframe with more informative player stats
def build_history_dataframe(player_history, variant_id, streamlit=False):
    
    # Option to view 'streamlit' dataframe, which includes pertinent
    # information but excludes all stats for modeling
    if streamlit == True:
        vdf_columns = ['Gamertag','TotalTimePlayed','K/D','Accuracy','WinRate']
        vdf = pd.DataFrame(columns = vdf_columns)
    else:
        stat_list = ['Gamertag', 'TotalKills', 'TotalHeadshots', 'TotalWeaponDamage', 'TotalShotsFired',
                    'TotalShotsLanded', 'TotalMeleeKills', 'TotalMeleeDamage', 'TotalAssassinations',
                    'TotalGroundPoundKills', 'TotalGroundPoundDamage', 'TotalShoulderBashKills',
                    'TotalShoulderBashDamage', 'TotalGrenadeDamage', 'TotalPowerWeaponKills',
                    'TotalPowerWeaponDamage', 'TotalPowerWeaponGrabs', 'TotalPowerWeaponPossessionTime',
                    'TotalDeaths', 'TotalAssists', 'TotalGamesCompleted', 'TotalGamesWon',
                    'TotalGamesLost', 'TotalGamesTied', 'TotalTimePlayed','TotalGrenadeKills']
        vdf = pd.DataFrame(columns = stat_list)
    
    # Set coutner variable
    i = 0
    # Loop the goes through each player in the player history JSON
    for player in player_history['Results']:
        
        # Loop that goes through each Arena Game Base Variant and locates
        # the details specific to the game vase variant of the match
        for variant in player['Result']['ArenaStats']['ArenaGameBaseVariantStats']:
            if variant['GameBaseVariantId'] == variant_id:
                variant_stats = variant
        
        # Create empty dictionary where stats will be added
        variant_dic = {}
        
        # Streamlit option - calculates specifc features
        if streamlit == True:
            variant_dic['Gamertag'] = player_history['Results'][i]['Id']
            variant_dic['TotalTimePlayed']= isodate.parse_duration(variant_stats['TotalTimePlayed']).total_seconds() / 3600
            variant_dic['K/D'] = variant_stats['TotalKills'] / variant_stats['TotalDeaths']
            variant_dic['Accuracy'] = variant_stats['TotalShotsLanded'] / variant_stats['TotalShotsFired']
            variant_dic['WinRate'] = variant_stats['TotalGamesWon'] / variant_stats['TotalGamesLost']
            vdf = vdf.append(variant_dic, True)
            i += 1
        
        # Modeling option - includes all features but does not yet calculate
        else:
            variant_dic['Gamertag'] = player_history['Results'][i]['Id']
            variant_dic['TotalTimePlayed']= isodate.parse_duration(variant_stats['TotalTimePlayed']).total_seconds() / 3600
            variant_dic['K/D'] = variant_stats['TotalKills'] / variant_stats['TotalDeaths']
            variant_dic['Accuracy'] = variant_stats['TotalShotsLanded'] / variant_stats['TotalShotsFired']
            variant_dic['WinRate'] = variant_stats['TotalGamesWon'] / variant_stats['TotalGamesLost']
            
            # Loop that appends all stats to variant dic
            for stat in stat_list[1:]:    
                variant_dic[stat] = variant_stats[stat]
            
            # Parsing ISO duration times
            variant_dic['TotalTimePlayed']= isodate.parse_duration(variant_stats['TotalTimePlayed']).total_seconds() / 3600
            variant_dic['TotalPowerWeaponPossessionTime']= isodate.parse_duration(variant_stats['TotalPowerWeaponPossessionTime']).total_seconds() / 3600
#             vdf = vdf.append(variant_dic, True)
#             i += 1
            
            # Per game stats
            per_game_stat_list = ['TotalKills', 'TotalHeadshots', 'TotalWeaponDamage', 
                                  'TotalShotsFired', 'TotalShotsLanded', 'TotalMeleeKills', 
                                  'TotalMeleeDamage', 'TotalAssassinations', 'TotalGroundPoundKills', 
                                  'TotalGroundPoundDamage', 'TotalShoulderBashKills', 
                                  'TotalShoulderBashDamage', 'TotalGrenadeDamage', 'TotalPowerWeaponKills', 
                                  'TotalPowerWeaponDamage', 'TotalPowerWeaponGrabs', 
                                  'TotalPowerWeaponPossessionTime', 'TotalDeaths', 'TotalAssists', 
                                  'TotalGrenadeKills']
            
            for stat in per_game_stat_list:
                per_game_stat_string = stat.replace('Total', '')
                per_game_stat_string = f'{per_game_stat_string}PerGame'
                variant_dic[per_game_stat_string] = variant_dic[stat] / variant_dic['TotalGamesCompleted']
            
            
            vdf = vdf.append(variant_dic, True)
            i += 1
            
    # Return the streamlit or modeling dataframe
    return vdf
    
# build_history_dataframe(player_history, '1571fdac-e0b4-4ebc-a73a-6e13001b71d3', streamlit=False)
# df = recent_match_stats('Drymander', back_count=0)
# df

Overwriting build_history_dataframe.py


## Decode Column

In [67]:
# %%writefile 'decode_column.py'

# This function will convert codes provided by the API into a readable format
def decode_column(df, column, api_dict):
    
    # Empty list of decoded values
    decoded_list = []
    
    # Loop through each row
    for row in df[column]:
        i = 0
        
        # Loop through API dictionary
        for item in api_dict:
            
            # If code found, append it to list
            if item['id'] == row:
                name = item['name']
                decoded_list.append(name)
            
            # Otherwise keep searching until found
            else:
                i += 1
    
    # Return decoded list
    return decoded_list

Writing decode_column.py


## Decode Maps

In [68]:
# %%writefile 'decode_maps.py'

# This function will convert maps to readable format
def decode_maps(df, column, api_dict):
    decoded_list = []
    
    # Loop through each row
    for row in df[column]:
        i = 0
        
        # Creating map_count variable
        map_count = len(api_dict)
        
        # For each item in API dictionary
        for item in api_dict:
            
            # If map cannot be found, name 'Custom Map'
            if (i+1) == map_count:
                name = 'Custom Map'
                decoded_list.append(name)
            
            # If found, assign value to code
            elif item['id'] == row:
                name = item['name']
                decoded_list.append(name)
            
            # Otherwise keep looping
            else:
                i += 1
    
    # Return decoded list
    return decoded_list

Writing decode_maps.py


## Load Pickle Files

GameBaseVariantId, PlaylistId, and MapVariantId

In [69]:
# no write file here

# Loading GameBaseVariantId metadata dictionary pulled from API
with open('GameBaseVariantId.pkl', 'rb') as GameBaseVariantId_pickle:
    GameBaseVariantId_dic = pickle.load(GameBaseVariantId_pickle)

# Loading PlaylistId metadata dictionary pulled from API
with open('PlaylistId_dic.pkl', 'rb') as PlaylistId_dic_pickle:
    PlaylistId_dic = pickle.load(PlaylistId_dic_pickle)

# Loading map_list metadata dictionary pulled from API
with open('map_list.pkl', 'rb') as map_list_pickle:
    map_list = pickle.load(map_list_pickle)

# # Decode columsn with using our decode functions
# df['GameBaseVariantId'] = decode_column(df, 'GameBaseVariantId', GameBaseVariantId_dic)    
# df['PlaylistId'] = decode_column(df, 'PlaylistId', PlaylistId_dic)
# df['MapVariantId'] = decode_maps(df, 'MapVariantId', map_list)

# df[['GameBaseVariantId', 'PlaylistId', 'MapVariantId']].head(3)

## Master Function - Recent Match Stats

In [170]:
# %%writefile 'recent_match_stats.py'

# Function that combines all functions above to go through each step to
# Get the match dataframe
def recent_match_stats(gamertag, back_count=0):
    
    # Pull the match result as JSON from API
    match_results = pull_recent_match(gamertag, back_count, explore=False)
    
    # Build the base dataframe
    base_df = build_base_dataframe(match_results, gamertag=gamertag)
    
    # Convert dates
    base_df['Date'] = base_df['Date'].dt.strftime('%B, %d %Y')
    
    # Decode GameBaseVariantId, PlaylistId, and MapVariantId
    base_df['GameBaseVariantId'] = decode_column(base_df, 'GameBaseVariantId', GameBaseVariantId_dic)    
    base_df['PlaylistId'] = decode_column(base_df, 'PlaylistId', PlaylistId_dic)
    base_df['MapVariantId'] = decode_maps(base_df, 'MapVariantId', map_list)
    
    # Sleep for 1.01 seconds to avoid issues with API
    time.sleep(1.01)
    
    # Create playerlist for player history API call
    player_list = get_player_list(base_df)
    
    # Call API to get player history JSON
    player_history = get_player_history(base_df)
    
    # Build base player stats dataframe based on player history API call
    history_df = build_history_dataframe(player_history, match_results['GameBaseVariantId'])
    
    # Merge the base dataframe and stats dataframe
    full_stats_df = pd.merge(base_df, history_df, how='inner', on = 'Gamertag')
    
    return full_stats_df

# Show full dataframe for match
# df = recent_match_stats('Drymander', back_count=0)
# df.to_csv('data/match.csv')

# Graphs

In [71]:
import plotly.express as px
# df = df
fig = px.bar(df, x='TotalTimePlayed', y='Gamertag', orientation='h')
fig.show()

In [72]:
df['TotalHeadshots'].dtypes
type(df['TotalHeadshots'].iloc[0]) == int
# type(df['TotalTimePlayed'].iloc[0]) == np.float64

True

In [172]:
df = recent_match_stats('Drymander', 0)
df.to_csv('data/match.csv')
df

,Date,MatchId,GameBaseVariantId,PlaylistId,MapVariantId,DNF,TeamId,PlayerTeam,Winner,TeamColor,Gamertag,SpartanRank,PrevTotalXP,TotalKills,TotalHeadshots,TotalWeaponDamage,TotalShotsFired,TotalShotsLanded,TotalMeleeKills,TotalMeleeDamage,TotalAssassinations,TotalGroundPoundKills,TotalGroundPoundDamage,TotalShoulderBashKills,TotalShoulderBashDamage,TotalGrenadeDamage,TotalPowerWeaponKills,TotalPowerWeaponDamage,TotalPowerWeaponGrabs,TotalPowerWeaponPossessionTime,TotalDeaths,TotalAssists,TotalGamesCompleted,TotalGamesWon,TotalGamesLost,TotalGamesTied,TotalTimePlayed,TotalGrenadeKills,Accuracy,AssassinationsPerGame,AssistsPerGame,DeathsPerGame,GrenadeDamagePerGame,GrenadeKillsPerGame,GroundPoundDamagePerGame,GroundPoundKillsPerGame,HeadshotsPerGame,K/D,KillsPerGame,MeleeDamagePerGame,MeleeKillsPerGame,PowerWeaponDamagePerGame,PowerWeaponGrabsPerGame,PowerWeaponKillsPerGame,PowerWeaponPossessionTimePerGame,ShotsFiredPerGame,ShotsLandedPerGame,ShoulderBashDamagePerGame,ShoulderBashKillsPerGame,WeaponDamagePerGame,WinRate
0,"July, 26 2021",4d10fbcf-c1ef-4bae-a3da-0000f66ff934,Slayer,Super Fiesta Party,Mercy,0.0,1.0,Enemy,Victory,Blue,iZK8182794,144,7112419,23037,4597,2.779111e+06,563356,187286,1554,143121.249992,251,1,210.000000,122,16116.018838,161003.670295,12642,1.653321e+06,312,87.395062,23513,6358,1813,840,972,1,220.031596,1101,0.332447,0.138445,3.506895,12.969112,88.805113,0.607281,0.115830,0.000552,2.535576,0.979756,12.706564,78.941671,0.857143,911.925692,0.172090,6.972973,0.048205,310.731384,103.301710,8.889144,0.067292,1532.880032,0.864198
1,"July, 26 2021",4d10fbcf-c1ef-4bae-a3da-0000f66ff934,Slayer,Super Fiesta Party,Mercy,0.0,1.0,Enemy,Victory,Blue,PlagueDoctor924,141,6539824,16044,3057,1.837408e+06,376067,126394,1680,233531.102305,519,42,4890.996887,274,39753.824627,37010.256863,7489,9.683606e+05,513,65.560384,19750,4848,1625,544,1080,1,196.045298,277,0.336094,0.319385,2.983385,12.153846,22.775543,0.170462,3.009844,0.025846,1.881231,0.812354,9.873231,143.711448,1.033846,595.914224,0.315692,4.608615,0.040345,231.425846,77.780923,24.463892,0.168615,1130.712601,0.503704
2,"July, 26 2021",4d10fbcf-c1ef-4bae-a3da-0000f66ff934,Slayer,Super Fiesta Party,Mercy,0.0,1.0,Enemy,Victory,Blue,HxC Shingon,113,4185606,14922,8015,2.058423e+06,326786,120334,531,73529.221862,81,22,2986.530644,11,2378.248744,127685.411140,5934,8.637453e+05,291,50.631515,13820,4859,1311,619,692,0,160.517823,474,0.368235,0.061785,3.706331,10.541571,97.395432,0.361556,2.278055,0.016781,6.113654,1.079740,11.382151,56.086363,0.405034,658.844587,0.221968,4.526316,0.038621,249.264683,91.787948,1.814072,0.008391,1570.116715,0.894509
3,"July, 26 2021",4d10fbcf-c1ef-4bae-a3da-0000f66ff934,Slayer,Super Fiesta Party,Mercy,0.0,1.0,Enemy,Victory,Blue,DrDiggler74,133,5690840,19583,5582,2.561268e+06,492743,175670,1041,134487.233584,178,0,0.000000,57,12871.061153,301902.114653,9673,1.274141e+06,145,78.605749,19944,5863,1773,857,915,1,216.639274,1176,0.356514,0.100395,3.306825,11.248731,170.277560,0.663283,0.000000,0.000000,3.148336,0.981899,11.045121,75.852924,0.587140,718.635744,0.081782,5.455725,0.044335,277.914834,99.080654,7.259482,0.032149,1444.595482,0.936612
4,"July, 26 2021",4d10fbcf-c1ef-4bae-a3da-0000f66ff934,Slayer,Super Fiesta Party,Mercy,0.0,0.0,Player,Defeat,Red,Drymander,148,15620337,28819,2160,3.829023e+06,756773,242580,1482,161700.288293,209,35,5230.651564,100,19190.688572,129347.927789,23390,3.114434e+06,140,183.842909,28037,5921,2526,1258,1268,0,299.825284,459,0.320545,0.082740,2.344022,11.099367,51.206622,0.181710,2.070725,0.013856,0.855107,1.027892,11.408947,64.014366,0.586698,1232.951094,0.055424,9.259699,0.072780,299.593428,96.033254,7.597264,0.039588,1515.844507,0.992114
5,"July, 26 2021",4d10fbcf-c1ef-4bae-a3da-0000f66ff934,Slayer,Super Fiesta Party,Mercy,0.0,0.0,Player,Defeat,Red,MarySueMack,148,14325326,44617,8212,5.690563e+06,927912,372361,2746,350708.783403,661,22,5127.313930,594,119023.092286,143133.918647,27630,3.651009e+06,61,220.862244,417

# Compare Stats Plot Function

In [144]:
# %%writefile 'compare_stat.py'



import plotly.graph_objects as go
# from plotly.subplots import make_subplots
from plotly.subplots import make_subplots
def compare_stat(df, column_name):
    
#     layout = go.Layout(
#         margin=go.layout.Margin(
#             l=100, #left margin
#             r=0, #right margin
#             b=0, #bottom margin
#             t=0))  #top margin
    
    df = df.round(2)
    # Separate player and enemy teams
    df_player = df.loc[df['PlayerTeam'] == 'Player']
    df_enemy = df.loc[df['PlayerTeam'] == 'Enemy']

    # Sort total time played by descending
    df_player = df_player.sort_values(by=[column_name])
    df_enemy = df_enemy.sort_values(by=[column_name])

    # Assign player / enemy colors
    if df_player['TeamColor'].iloc[0] == 'Blue':
        player_color = 'Blue'
        enemy_color = 'Red'
    else:
        player_color = 'Red'
        enemy_color = 'Blue'
    
    # Make subplot and X axis range
    fig = make_subplots(rows=2, cols=1, subplot_titles=[f'Player Team - {column_name}', 
                                                        f'Enemy Team - {column_name}'],
                       vertical_spacing = 0.12)
    x_range = df[column_name].max()
    
    # Player team sub plot
    fig.add_trace(go.Bar(
                x=df_player[column_name],
                y=df_player['Gamertag'],
                orientation='h',
                text=df_player[column_name],
                textposition='auto',
                marker_color=player_color),
                    row=1, col=1)
    fig.update_xaxes(range=[0, x_range], row=1, col=1)
    
    # Enemy team sub plot
    fig.add_trace(go.Bar(
                x=df_enemy[column_name],
                y=df_enemy['Gamertag'],
                orientation='h',
                text=df_enemy[column_name],
                textposition='auto',
                marker_color=enemy_color),
                    row=2, col=1)
    fig.update_xaxes(range=[0, x_range], row=2, col=1)
    fig.update_yaxes(automargin=True)
    fig['layout'].update(margin=dict(l=125,r=50,b=20,t=30))
    fig['layout'].update(showlegend=False)
#     fig.update_layout(title_text='test')
    return fig

compare_stat(df, 'TotalHeadshots')

Overwriting compare_stat.py


In [154]:
df

,Date,MatchId,GameBaseVariantId,PlaylistId,MapVariantId,DNF,TeamId,PlayerTeam,Winner,TeamColor,Gamertag,SpartanRank,PrevTotalXP,TotalKills,TotalHeadshots,TotalWeaponDamage,TotalShotsFired,TotalShotsLanded,TotalMeleeKills,TotalMeleeDamage,TotalAssassinations,TotalGroundPoundKills,TotalGroundPoundDamage,TotalShoulderBashKills,TotalShoulderBashDamage,TotalGrenadeDamage,TotalPowerWeaponKills,TotalPowerWeaponDamage,TotalPowerWeaponGrabs,TotalPowerWeaponPossessionTime,TotalDeaths,TotalAssists,TotalGamesCompleted,TotalGamesWon,TotalGamesLost,TotalGamesTied,TotalTimePlayed,TotalGrenadeKills,Accuracy,AssassinationsPerGame,AssistsPerGame,DeathsPerGame,GrenadeDamagePerGame,GrenadeKillsPerGame,GroundPoundDamagePerGame,GroundPoundKillsPerGame,HeadshotsPerGame,K/D,KillsPerGame,MeleeDamagePerGame,MeleeKillsPerGame,PowerWeaponDamagePerGame,PowerWeaponGrabsPerGame,PowerWeaponKillsPerGame,PowerWeaponPossessionTimePerGame,ShotsFiredPerGame,ShotsLandedPerGame,ShoulderBashDamagePerGame,ShoulderBashKillsPerGame,WeaponDamagePerGame,WinRate
0,"July, 22 2021",2eecaf00-ba9f-432e-b144-bc1c90b29430,Slayer,Super Fiesta Party,Plaza,0.0,0.0,Enemy,Victory,Red,Dimension vv,138,6187181,18530,9433,1.828544e+06,265430,106770,981,1.655610e+05,569,106,12037.056124,217,28556.529271,8.783789e+04,6366,7.964073e+05,457,47.396104,18288,4075,1548,723,825,0,177.430622,515,0.402253,0.367571,2.632429,11.813953,56.742822,0.332687,7.775876,0.068475,6.093669,1.013233,11.970284,106.951552,0.633721,514.475032,0.295220,4.112403,0.030618,171.466408,68.972868,18.447370,0.140181,1181.230006,0.876364
1,"July, 22 2021",2eecaf00-ba9f-432e-b144-bc1c90b29430,Slayer,Super Fiesta Party,Plaza,0.0,0.0,Enemy,Victory,Red,Nefarious,144,7508121,22619,9779,2.341174e+06,301329,132159,1768,1.827042e+05,498,183,16618.659790,347,26330.779343,1.470351e+05,8724,1.057324e+06,520,56.212851,20930,4852,1726,803,923,0,195.564511,758,0.438587,0.288528,2.811124,12.126304,85.188380,0.439166,9.628424,0.106025,5.665701,1.080698,13.104867,105.854090,1.024334,612.586598,0.301275,5.054461,0.032568,174.582271,76.569525,15.255376,0.201043,1356.416079,0.869989
2,"July, 22 2021",2eecaf00-ba9f-432e-b144-bc1c90b29430,Slayer,Super Fiesta Party,Plaza,0.0,0.0,Enemy,Victory,Red,SourStarfish28,143,6823910,19021,1565,2.442232e+06,346960,132726,522,5.683225e+04,99,12,2313.328398,71,10473.208076,3.920214e+04,16021,2.080327e+06,16,124.868726,18264,3729,1483,730,752,1,193.553819,212,0.382540,0.066757,2.514498,12.315577,26.434346,0.142953,1.559898,0.008092,1.055293,1.041448,12.826028,38.322488,0.351989,1402.782982,0.010789,10.803102,0.084200,233.958193,89.498314,7.062177,0.047876,1646.818840,0.970745
3,"July, 22 2021",2eecaf00-ba9f-432e-b144-bc1c90b29430,Slayer,Super Fiesta Party,Plaza,0.0,0.0,Enemy,Victory,Red,Nottheeddu,150,27262125,15350,4305,2.086481e+06,396248,136780,1021,1.581637e+05,314,60,10741.944481,84,16998.106466,1.828776e+05,7468,1.005274e+06,124,60.039112,14490,4623,1367,655,711,1,164.625542,737,0.345188,0.229700,3.381858,10.599854,133.780261,0.539137,7.858043,0.043892,3.149232,1.059351,11.228969,115.701352,0.746891,735.386701,0.090710,5.463058,0.043920,289.866862,100.058522,12.434606,0.061448,1526.321137,0.921238
4,"July, 22 2021",2eecaf00-ba9f-432e-b144-bc1c90b29430,Slayer,Super Fiesta Party,Plaza,0.0,1.0,Player,Defeat,Blue,Drymander,148,15539108,28622,2148,3.804711e+06,752121,241046,1473,1.607738e+05,208,35,5230.651564,100,19095.688572,1.279060e+05,23221,3.094004e+06,140,182.570881,27853,5890,2510,1252,1258,0,297.931417,452,0.320488,0.082869,2.346614,11.096813,50.958578,0.180080,2.083925,0.013944,0.855777,1.027609,11.403187,64.053293,0.586853,1232.670745,0.055777,9.251394,0.072737,299.649801,96.034263,7.607844,0.039841,1515.821047,0.995231
5,"July, 22 2021",2eecaf00-ba9f-432e-b144-bc1c90b29430,Slayer,Super Fiesta Party,Plaza,0.0,1.0,Player,Defeat,Blue,cryptolockerSD,151,47129356,53899,6851,7.018691e+06,1244500,439444,4114,4.847869e+05,829,184,36020.682724,98,22007.556794,6.860665e+05,33654,4.379956e+06,691,257.554

In [155]:
gamertag

df_outcome = df.loc[df['Gamertag'] == gamertag]

gamebasevariantid = df_outcome['GameBaseVariantId'].iloc[0]
playlistid = df_outcome['PlaylistId'].iloc[0]
map_name = df_outcome['MapVariantId'].iloc[0]
if df_outcome['Winner'].iloc[0] == 'Victory':
    print('Congrats on your victory!')
    
print(f"Showing stats for your match on {df_outcome['Date'].iloc[0]}")
print(f'Game Mode - {gamebasevariantid}')
print(f'Playlist - {playlistid}')
print(f'Map - {map_name}')


Showing stats for your match on July, 22 2021
Game Mode - Slayer
Playlist - Super Fiesta Party
Map - Plaza


In [ ]:
Index(['Date', 'MatchId', 'GameBaseVariantId', 'PlaylistId', 'MapVariantId',
       'DNF', 'TeamId', 'PlayerTeam', 'Winner', 'TeamColor', 'Gamertag',
       '', '', '', '',
       '', '', '',
       '', '', '',
       '', '',
       '', '',
       '', '', '',
       '', '',
       '', '', '', '',
       '', '', '',
       '', '', '',
       '', '', '',
       '', '',
       '', '', '', '',
       '', '', '',
       '', '',
       '', '',
       '', '',
       '', '', ''],

# Unused Code

In [ ]:
# %%writefile compare_stat.py

df = recent_match_stats('Drymander', 0)

import plotly.graph_objects as go
# from plotly.subplots import make_subplots
from plotly.subplots import make_subplots
def compare_stat(df, column_name):

    # Separate player and enemy teams
    df_player = df.loc[df['PlayerTeam'] == 'Player']
    df_enemy = df.loc[df['PlayerTeam'] == 'Enemy']

    # Sort total time played by descending
    df_player = df_player.sort_values(by=[column_name])
    df_enemy = df_enemy.sort_values(by=[column_name])

    # Assign player / enemy colors
    if df_player['TeamColor'].iloc[0] == 'Blue':
        player_color = 'Blue'
        enemy_color = 'Red'
    else:
        player_color = 'Red'
        enemy_color = 'Blue'
    
    # Make subplot and X axis range
    fig = make_subplots(rows=2, cols=1, subplot_titles=['Player Team', 'Enemy Team'])
    x_range = df[column_name].max()
    
    # Text
    if type(df[column_name].iloc[0]) == int:
        player_text = df_player[column_name],
        enemy_text = df_enemy[column_name],
    else:
        player_text = df_player[column_name].round(2),
        enemy_text = df_enemy[column_name].round(2),
    # Player team sub plot
    fig.add_trace(go.Bar(
                x=df_player[column_name],
                y=df_player['Gamertag'],
                orientation='h',
                text=player_text,
                textposition='auto',
                marker_color=player_color),
                    row=1, col=1)
    fig.update_xaxes(range=[0, x_range], row=1, col=1)
    
    # Enemy team sub plot
    fig.add_trace(go.Bar(
                x=df_enemy[column_name],
                y=df_enemy['Gamertag'],
                orientation='h',
                text=enemy_text,
                textposition='auto',
                marker_color=enemy_color),
                    row=2, col=1)
    fig.update_xaxes(range=[0, x_range], row=2, col=1)
    fig.update_layout(title_text='test')
    return fig

compare_stat(df, 'TotalHeadshots')

## Build history dataframe before removing streamlit

In [ ]:
# %%writefile 'build_history_dataframe.py'

# Function to build secondary dataframe with more informative player stats
def build_history_dataframe(player_history, variant_id, streamlit=False):
    
    # Option to view 'streamlit' dataframe, which includes pertinent
    # information but excludes all stats for modeling
    if streamlit == True:
        vdf_columns = ['Gamertag','TotalTimePlayed','K/D','Accuracy','WinRate']
        vdf = pd.DataFrame(columns = vdf_columns)
    else:
        stat_list = ['Gamertag', 'TotalKills', 'TotalHeadshots', 'TotalWeaponDamage', 'TotalShotsFired',
                    'TotalShotsLanded', 'TotalMeleeKills', 'TotalMeleeDamage', 'TotalAssassinations',
                    'TotalGroundPoundKills', 'TotalGroundPoundDamage', 'TotalShoulderBashKills',
                    'TotalShoulderBashDamage', 'TotalGrenadeDamage', 'TotalPowerWeaponKills',
                    'TotalPowerWeaponDamage', 'TotalPowerWeaponGrabs', 'TotalPowerWeaponPossessionTime',
                    'TotalDeaths', 'TotalAssists', 'TotalGamesCompleted', 'TotalGamesWon',
                    'TotalGamesLost', 'TotalGamesTied', 'TotalTimePlayed','TotalGrenadeKills']
        vdf = pd.DataFrame(columns = stat_list)
    
    # Set coutner variable
    i = 0
    # Loop the goes through each player in the player history JSON
    for player in player_history['Results']:
        
        # Loop that goes through each Arena Game Base Variant and locates
        # the details specific to the game vase variant of the match
        for variant in player['Result']['ArenaStats']['ArenaGameBaseVariantStats']:
            if variant['GameBaseVariantId'] == variant_id:
                variant_stats = variant
        
        # Create empty dictionary where stats will be added
        variant_dic = {}
        
        # Streamlit option - calculates specifc features
        if streamlit == True:
            variant_dic['Gamertag'] = player_history['Results'][i]['Id']
            variant_dic['TotalTimePlayed']= isodate.parse_duration(variant_stats['TotalTimePlayed']).total_seconds() / 3600
            variant_dic['K/D'] = variant_stats['TotalKills'] / variant_stats['TotalDeaths']
            variant_dic['Accuracy'] = variant_stats['TotalShotsLanded'] / variant_stats['TotalShotsFired']
            variant_dic['WinRate'] = variant_stats['TotalGamesWon'] / variant_stats['TotalGamesLost']
            vdf = vdf.append(variant_dic, True)
            i += 1
        
        # Modeling option - includes all features but does not yet calculate
        else:
            variant_dic['Gamertag'] = player_history['Results'][i]['Id']
            
            # Loop that appends all stats to variant dic
            for stat in stat_list[1:]:    
                variant_dic[stat] = variant_stats[stat]
            
            # Parsing ISO duration times
            variant_dic['TotalTimePlayed']= isodate.parse_duration(variant_stats['TotalTimePlayed']).total_seconds() / 3600
            vdf = vdf.append(variant_dic, True)
            i += 1
    
    # Return the streamlit or modeling dataframe
    return vdf
    
# build_history_dataframe(player_history, '1571fdac-e0b4-4ebc-a73a-6e13001b71d3', streamlit=False)